In [22]:
import numpy as np
import random

alfa = 0.2
gama = 0.95
epsilon = 0.1

grade_recompensa = np.full((4, 4), -1)
grade_recompensa[0, 2] = -20
grade_recompensa[3, 1] = -20
grade_recompensa[2, 1] = -5
grade_recompensa[1, 3] = -5
grade_recompensa[3, 3] = 20

estados_terminais = {(1,3), (4,2), (4,4)}

def estado_para_indice(estado):
    linha, coluna = estado
    return (linha - 1) * 4 + (coluna - 1)

acoes = {
    0: (1, 0),
    1: (0, 1),
    2: (-1, 0),
    3: (0, -1)
}

def passo(estado, acao):
    linha, coluna = estado
    dl, dc = acoes[acao]
    nova_linha = linha + dl
    nova_coluna = coluna + dc
    if nova_linha < 1 or nova_linha > 4 or nova_coluna < 1 or nova_coluna > 4:
        return estado, -1, False
    novo_estado = (nova_linha, nova_coluna)
    recompensa = grade_recompensa[nova_linha - 1, nova_coluna - 1]
    terminou = novo_estado in estados_terminais
    return novo_estado, recompensa, terminou

def escolher_acao(indice_estado, Q, modo="epsilon"):
    if modo == "greedy":
        return int(np.argmax(Q[indice_estado]))
    if modo == "random":
        return random.choice([0, 1, 2, 3])
    if modo == "manual":
        return int(input())
    if random.random() < epsilon:
        return random.choice([0, 1, 2, 3])
    return int(np.argmax(Q[indice_estado]))

Q = np.zeros((16, 4))

for linha in range(1,5):
    for coluna in range(1,5):
        idx = estado_para_indice((linha, coluna))
        if linha == 4:
            Q[idx,0] = -np.inf
        if coluna == 4:
            Q[idx,1] = -np.inf
        if linha == 1:
            Q[idx,2] = -np.inf
        if coluna == 1:
            Q[idx,3] = -np.inf

def executar_episodio(estado_inicial, Q, modo_acao="epsilon", imprimir=True):
    estado = estado_inicial
    passos = 0
    if imprimir:
        print(f"\n--- Novo episódio, estado inicial = {estado} ---")
    while True:
        indice = estado_para_indice(estado)
        acao = escolher_acao(indice, Q, modo_acao)

        novo_estado, recompensa, terminou = passo(estado, acao)
        indice_novo = estado_para_indice(novo_estado)

        valor_antigo = Q[indice, acao]
        Q[indice, acao] = Q[indice, acao] + alfa * (recompensa + gama * np.max(Q[indice_novo]) - Q[indice, acao])
        
        if imprimir:
            print(f"Passo {passos+1}: estado={estado}, ação={acao}, recompensa={recompensa}, próximo={novo_estado}")
            print(f"   Q antigo = {valor_antigo}, Q novo = {Q[indice, acao]}")
        estado = novo_estado
        passos += 1
        if terminou or passos >= 8:
            if imprimir:
                print(f"--- Episódio encerrado (terminou={terminou}, passos={passos}) ---")
            break
    return Q


In [23]:
episodios = [
    (2, 2),
    (3, 1),
    (1, 1),
    (1, 2),
    (1, 4),
    (1, 4),
    (2, 2),
    (3, 1),
    (1, 1)
]

for numero, estado_inicial in enumerate(episodios, start=1):
    print("\n==============================")
    print(f"EPISÓDIO {numero} — estado inicial: {estado_inicial}")
    Q = executar_episodio(estado_inicial, Q, modo_acao="greedy", imprimir=True)
    print("\nTabela Q após o episódio", numero)
    print(Q)


EPISÓDIO 1 — estado inicial: (2, 2)

--- Novo episódio, estado inicial = (2, 2) ---
Passo 1: estado=(2, 2), ação=0, recompensa=-5, próximo=(3, 2)
   Q antigo = 0.0, Q novo = -1.0
Passo 2: estado=(3, 2), ação=0, recompensa=-20, próximo=(4, 2)
   Q antigo = 0.0, Q novo = -4.0
--- Episódio encerrado (terminou=True, passos=2) ---

Tabela Q após o episódio 1
[[  0.   0. -inf -inf]
 [  0.   0. -inf   0.]
 [  0.   0. -inf   0.]
 [  0. -inf -inf   0.]
 [  0.   0.   0. -inf]
 [ -1.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0. -inf   0.   0.]
 [  0.   0.   0. -inf]
 [ -4.   0.   0.   0.]
 [  0.   0.   0.   0.]
 [  0. -inf   0.   0.]
 [-inf   0.   0. -inf]
 [-inf   0.   0.   0.]
 [-inf   0.   0.   0.]
 [-inf -inf   0.   0.]]

EPISÓDIO 2 — estado inicial: (3, 1)

--- Novo episódio, estado inicial = (3, 1) ---
Passo 1: estado=(3, 1), ação=0, recompensa=-1, próximo=(4, 1)
   Q antigo = 0.0, Q novo = -0.2
Passo 2: estado=(4, 1), ação=1, recompensa=-20, próximo=(4, 2)
   Q antigo = 0.0, Q novo = -4.